# Biases in the distribution of consecutive primes
In 2016 Lemke Oliver & Soundararajan published their statistical study of the last digits of consecutive primes.  They observed a bias (that is, an uneven distribution) among the results, over the first 100 million
primes.  This observed bias was heralded as "unexpected", but it reflects the relative populations of the associated gaps from the relative populations models $w_{g,1}(\lambda)$.

In this notebook we provide an interactive study of these biases, across different moduli.
The counts of gaps over the first $10^8$ primes are fixed around $\lambda=0.348$, and the counts over the first $10^{12}$ primes
are fixed around $\lambda=0.27$. 

From the graphs below we see that Lemke Oliver & Soundararajan's data is predicted by these population models.  The observed biases 
are consistent with the evolution of the populations of gaps across Eratosthenes sieve.  That is, their tabulated data is consistent
with the relative populations $W_r(\lambda)$ at the corresponding value of $\lambda$.

The models also show us that the biases are transient.  As Eratosthenes sieve continues the distribution of last digits of consecutive 
primes will be uniform across the possible ordered pairs.

For the underlying analysis, see the 2016 manuscript "On the last digits of consecutive primes" and the 2024 manuscript "Expected biases in the distribution of consecutive primes"

In [1]:
%reset -f

# This version uses pandas & matplotlib to provide interactive graphics
import pandas as pd
import numpy as np
from numpy.polynomial.polynomial import polyval
import array
import itertools
import matplotlib
%matplotlib inline
# matplotlib.use("nbagg")

import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
plt.ion

import gc
import psutil
import sys
import csv

In [2]:
pgaps23 = np.load('cG23.npy')
plen = pgaps23.size
maxp = 1 + np.sum(pgaps23)
print(f"{plen} gaps of sum {maxp}")

12283523 gaps of sum 223092871


In [3]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 2760.98 MB


In [4]:
# The matrix of coefficients lj is created in the '02' notebook.  Here we read the result from file.
# These coefficients include the alternating signs
ljmat = np.load('lj37.npy')
ljmat.shape
ljmat[0:3,0:5]

array([[ 1.        ,  1.        ,  2.        ,  1.        ,  1.33333333],
       [-0.        , -0.        , -0.6513022 , -0.6513022 , -0.9769533 ],
       [ 0.        ,  0.        ,  0.        ,  0.07157346,  0.14314691]])

In [5]:
# create an array of lambda values
lampar = np.arange(0,1,0.0005)
lampar = np.square(lampar) # These curves have more shape as lambda gets smaller, so we adjust the sampled values

In [6]:
# calculate values for the curves at the lambda values
# Note: we have to handle the model for g=82 separately, as a boundary case
wgpk = np.zeros((lampar.size, 41))
i = 0
while (i < 41):
    wgpk[:,i] = polyval(lampar, ljmat[:,i])
    i += 1


In [7]:
# Develop a master color dictionary - by residue class
# we set colors by family, as determined by the residue of the gap
Wmastercolordict={ '0': '#040404', '2':'#FF0000', '4':'#CCCC00', '6':'#0000FF', '8':'#00CC00',
                 '10':'#0064AA','12':'#1E90FF', '14':'#FF9A00','16':'#DC143C', '18':'#000080',
                 '20':'#008000','22':'#0000CD', '24':'#4169E1', '26':'#0000CD', '28':'#FF8000'}

In [8]:
# read in the asymptotic values w_g for gaps g <= 510510
# index:  g=2*(i+1)
wginf = np.load('wginf510510.npy')

In [9]:
def draw_modwg(xmax,ymin,modulus:int):
    # plotting the curves
    plt.clf()
    fig, ax = plt.subplots()
    fig.set_size_inches(12,9)

    # calculate maxgap for this modulus, to keep the teams fair
    modint = int(modulus)
    maxgap = int(np.floor(82/modint))
    maxgap = maxgap * modint
    
    ax.set_title(f"Teams $W_r(p_k^\#)$ mod {modint}, for $p_0=37$, gaps $g <$ {maxgap}, & $W_r(\infty)$ for gaps up to 510510")

    # set gridlines for x-axis (lambda)
    if (xmax < 0.05):
        marklampar = 0.001
    elif (xmax < 0.3):
        marklampar = 0.01
    elif (xmax < 0.75):
        marklampar = 0.05
    else:
        marklampar = 0.1
    ax.set_xticks(np.arange(0,xmax,marklampar))
    ax.grid(axis='x', color='#080408', lw=0.0625, markevery=marklampar)
    ax.grid(axis='y', color='#A9A9A9', lw=0.0625 )
    ax.set_xlim(0,xmax)
    ax.set_ylim(ymin,2.75)

    # form teams W_r
    maxrdex = int(modint/2)
    Nr = np.zeros((lampar.size, maxrdex))
    Wr = np.zeros((lampar.size, maxrdex))
    i=0
    gap = 2
    while (gap <= maxgap):
        r= gap % modint
        rdex = int(r / 2)
        Nr[:, rdex] = Nr[:, rdex]+ wgpk[:,i]
        i += 1
        gap += 2

    i=0
    while (i < maxrdex):
        Wr[:,i] = np.divide(Nr[:,i], Nr[:,1])
        i +=1

    # estimate the asymptotic values for W_r using wginf up to g=510510
    Wrinf = np.zeros(maxrdex)
    # reset maxj for 510510
    maxj = int(np.floor(510510/modint) * (modint/2))
#    print(f"modint {modint} maxj {maxj} maxrdex {maxrdex}")
    i=0
    while (i < maxrdex):
        j = (i-1) % maxrdex
        while (j < maxj):
            Wrinf[i] = Wrinf[i]+wginf[j]
            j += int(modint/2)
        i += 1

    i=0
    while (i < maxrdex):
        if (i != 1):
            Wrinf[i] = Wrinf[i]/Wrinf[1]
        i += 1
    Wrinf[1] = 1
    
    # plot the teams
    i=0
    while (i < maxrdex):
        rval = (2*i) % modint
        rstr=str(rval)
        winfstr = f"{Wrinf[i]:.3f}"
        wgcolor = Wmastercolordict[rstr]
        ax.plot(lampar, Wr[:,i], color=wgcolor, lw=1.5, label='r='+rstr+' Winf='+winfstr)
        i += 1

    # displaying the legend
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, ncol=3)

    # mark the computational sections at 10^8 primes and 10^12 primes
    secAx = np.array((0.348,0.348))
    secy = np.array((0.0, 2.4))
    ax.plot(secAx, secy, color='#A0A0A0', lw=1, linestyle='dashed')
    secBx = np.array((0.27,0.27))
    ax.plot(secBx, secy, color='#A0A0A0', lw=1, linestyle='dashed')
    
    plt.show()

# widgets for interacting with the graph of the w_g
xmaxSelect = widgets.SelectionSlider(options=[0.01,0.05,0.1,0.15,0.2,0.3,0.5,0.75,1.0],
                                      value=1.0, description="max lambda", layout=widgets.Layout(width='60%'), disabled=False)
yminSelect = widgets.SelectionSlider(options=[0.0, 0.1, 0.25, 0.5, 0.75],
                                      value=0.0, description="min w_g", layout=widgets.Layout(width='60%'), disabled=False)
modulusSelect = widgets.SelectionSlider(options=[4,6,8,10,12,14,16,18,20,30],
                                      value=10, description="Modulus", layout=widgets.Layout(width='60%'), disabled=False)

interact(draw_modwg, xmax=xmaxSelect, ymin=yminSelect, modulus=modulusSelect)


interactive(children=(SelectionSlider(description='max lambda', index=8, layout=Layout(width='60%'), options=(…

<function __main__.draw_modwg(xmax, ymin, modulus: int)>

## The plots of $W_r(\lambda)$
In the above graph we plot the relative populations $W_r(\lambda)$ of the teams of gaps for each residue class $r \bmod M$, for
modulus $M$.
$$ W_r(\lambda) \; = \; N_r(\lambda) / N_2(\lambda)$$
where
$$ N_r(\lambda) \; = \; \sum_{g = r \bmod M}^{g_{\rm max}} n_{g,1}(\lambda)$$
is the sum of the populations of the gaps $g \le g_{\rm max}$ in that residue class modulo $M$.

Currently we only have models for $2 \le g \le 82$.  So $g_{\rm max} \le 82$ and for each selection of $M$ we want an equal number of gaps on each team. 

The two dashed lines mark the values for $10^8$ and $10^{12}$ primes, $\lambda=0.348$ and $\lambda=0.27$ respectively, indicating the expected biases in calculations like those performed by Lemke-Oliver and Soundararajan.  If we look to the left where $\lambda=0$, we see that as $p \longrightarrow \infty$ the $W_r$ approach a distribution more consistent with the naive estimates based on numbers of ordered pairs of last digits.

Even without the complete models $w_{g,1}(\lambda)$ we can calculate the asymptotic values $w_{g,1}(\infty)$ for larger gaps. The asymptotic values in the legend are for gaps $g \le 510510=17^\#$ on each team $W_r$.